In [ ]:
from transformers import CamembertTokenizer, CamembertModel
import pandas as pd
import torch 
import numpy
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

<h1>Merge all train arrays</h1>

In [ ]:
train = pd.read_csv("./data/final/train.csv")

# Charger le tokenizer CamemBERT et le modèle CamemBERT
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

j = 9999
while j < 259999:
    # Définissez vos commentaires en tant que liste de chaînes
    comments = train["lemmatized"][j:j+1].to_list()

    # Tokenizez vos commentaires en utilisant le tokenizer CamemBERT
    input_ids = [tokenizer.encode(comment, return_tensors='pt', max_length=512) for comment in comments]

    # Obtenez les vecteurs de mots pour chaque commentaire en utilisant le modèle CamemBERT
    vectors = [model(input_id)[0][0][0].detach().numpy() for input_id in input_ids]

    numpy.save("./vector"+str(j+2), vectors)
    
    j = j + 10000
    print(j)

In [ ]:
import os

files = os.listdir("./data/vectors/train/")
arrays = []
for file in files:
    arrays.append(numpy.load("data/vectors/train/"+file))
result = numpy.vstack(arrays)

<h1>Test with 250000 comments</h1>

In [ ]:
train = pd.read_csv("./data/final/train.csv")
dev = pd.read_csv("./data/final/dev.csv")

In [ ]:
train_vectors = numpy.load("./data/vectors/train/train_250000.npy")
train_vectors = numpy.column_stack((train_vectors, train["sentimental_score"][0:250000].values))
numpy.save("./data/vectors/train_sentiment-250000",train_vectors)

dev_vectors = numpy.load("./data/vectors/dev/dev.npy")
dev_vectors = numpy.column_stack((dev_vectors, dev["sentimental_score"].values))
numpy.save("./data/vectors/dev_sentiment",dev_vectors)

train_y = train["note"][0:250000].astype("string")
dev_y = dev["note"].astype("string")

In [ ]:
clf = SGDClassifier(loss = "hinge", penalty = "l2")
clf.fit(train_vectors, train_y)

In [ ]:
y_pred = clf.predict(dev_vectors)
print(accuracy_score(dev_y,y_pred))